## VacationPy

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

Store Part I results from WeatherPy into DataFrame

In [11]:
# Load the csv exported in Part I to a DataFrame
path="../WeatherPy/data_sets/weather_data.csv"

# Read csv file to a dataframe
weather_df=pd.read_csv(path)
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Lubango,-14.9172,13.4925,58.32,59,10,2.15,AO,1661731107
1,Guéret,46.1667,1.8667,59.76,74,54,3.33,FR,1661731352
2,Ranong,9.9658,98.6348,78.66,88,100,1.92,TH,1661731352
3,Mataura,-46.1927,168.8643,46.47,59,3,1.90,NZ,1661731345
4,Pevek,69.7008,170.3133,41.95,74,7,4.00,RU,1661731353


 Humidity Heatmap

In [12]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations=weather_df[["Lat", "Lng"]]
humidity=weather_df["Humidity"]


In [13]:
# Add a heatmap layer to map
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=10, 
                                point_radius=1)

fig.add_layer(heat_layer)  
fig 

Figure(layout=FigureLayout(height='420px'))

Create new DataFrame fitting weather criteria

In [15]:
# Narrow down the cities to fit weather conditions:

# Max temp of 75 F but higher than 65 F
temp_df=weather_df[(weather_df["Max Temp"] > 65) & (weather_df["Max Temp"] < 75)]

# Wind speeds Less than 10 mph
wind_df=temp_df[temp_df["Wind Speed"] < 10 ]

# Zero clouds
clouds_df=wind_df[wind_df["Cloudiness"] == 0 ]

# Less than 50% humidity
humidity_df=clouds_df[clouds_df["Humidity"] < 50 ]

# Drop any rows with null values
perfect_weather_df=humidity_df.dropna()
perfect_weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,Ucar,40.5190,47.6542,71.47,46,0,2.64,AZ,1661731365
44,Göksun,38.0210,36.4973,65.05,38,0,4.03,TR,1661731367
74,Ardakān,32.3100,54.0175,71.82,10,0,5.28,IR,1661731377
157,Ondjiva,-17.0667,15.7333,65.35,20,0,6.78,AO,1661731408
166,Tabas,33.5959,56.9244,72.99,14,0,3.42,IR,1661731412
408,Ibipeba,-11.6408,-42.0111,69.58,47,0,5.93,BR,1661731501
485,Khoy,38.5504,44.9535,70.59,26,0,1.86,IR,1661731531
530,Astara,38.5000,48.6667,67.96,28,0,4.97,AZ,1661731547
554,Khuchni,41.9499,47.9485,67.60,43,0,4.52,RU,1661731557
557,Afşin,38.2477,36.9140,66.09,33,0,1.50,TR,1661731558


# Hotel Map

In [16]:
hotel_df=perfect_weather_df
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
40,Ucar,40.5190,47.6542,71.47,46,0,2.64,AZ,1661731365
44,Göksun,38.0210,36.4973,65.05,38,0,4.03,TR,1661731367
74,Ardakān,32.3100,54.0175,71.82,10,0,5.28,IR,1661731377
157,Ondjiva,-17.0667,15.7333,65.35,20,0,6.78,AO,1661731408
166,Tabas,33.5959,56.9244,72.99,14,0,3.42,IR,1661731412


In [17]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

#Set parameters to search for hotels within 5000 meters
base_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Hit the Google Places API for each city's coordinates
for index, row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]

# Change the location each iteration
    params["location"] = f"{lat}, {lng}"

    hotel_names=requests.get(base_url, params=params).json()

# For hotels that cannot be found with parameters, try- except
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]

    except:
        print("Could not find hotels near", row["City"])
        hotel_df.loc[index, "Hotel Name"] = " "

hotel_df

C:\Users\chris\AppData\Local\Temp\ipykernel_20036\3954780543.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


Could not find hotels near Ucar
Could not find hotels near Astara


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
40,Ucar,40.5190,47.6542,71.47,46,0,2.64,AZ,1661731365,
44,Göksun,38.0210,36.4973,65.05,38,0,4.03,TR,1661731367,EVİM APART OTEL
74,Ardakān,32.3100,54.0175,71.82,10,0,5.28,IR,1661731377,Ardakan Hotel
157,Ondjiva,-17.0667,15.7333,65.35,20,0,6.78,AO,1661731408,Águia Verde Hotel
166,Tabas,33.5959,56.9244,72.99,14,0,3.42,IR,1661731412,Amir hotel
408,Ibipeba,-11.6408,-42.0111,69.58,47,0,5.93,BR,1661731501,Pousada Velho chico
485,Khoy,38.5504,44.9535,70.59,26,0,1.86,IR,1661731531,Zomorod Hotel
530,Astara,38.5000,48.6667,67.96,28,0,4.97,AZ,1661731547,
554,Khuchni,41.9499,47.9485,67.60,43,0,4.52,RU,1661731557,"Restoranno-Gostinichnyy Kompleks ""Semeynyy Otd..."
557,Afşin,38.2477,36.9140,66.09,33,0,1.50,TR,1661731558,Gulumser Hatun Termal Otel & Spa


In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_names = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_names)

fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))